<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/2_%D0%BE_%D0%BA%D1%80%D0%B0%D1%83%D0%BB%D0%B8%D0%BD%D0%B3%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -qq install conllu
!pip -qq install graphviz
!pip -qq install ufal.udpipe


     |████████████████████████████████| 307kB 9.2MB/s 


In [2]:
# установим консольный браузер, пригодится
!apt -qq install lynx

The following packages were automatically installed and are no longer required:
  cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-curand-10-1 cuda-curand-dev-10-1
  cuda-cusolver-10-1 cuda-cusolver-dev-10-1 cuda-cusparse-10-1
  cuda-cusparse-dev-10-1 cuda-drivers cuda-npp-10-1 cuda-npp-dev-10-1
  cuda-nsight-10-1 cuda-nsight-compute-10-1 cuda-nsight-systems-10-1
  cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1 cuda-nvjpeg-10-1
  cuda-nvjpeg-dev-10-1 cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1 cuda-nvvp-10-1
  default-jre dkms freeglut3 freeglut3-dev keyboard-configuration libargon2-0
  libcap2 libcryptsetup12 libdevmapper1.02.1 libfontenc1 libgtk2.0-0
  libgtk2.0-common libip4tc0 libjansson4 libnvidia-cfg1-440
  libnvidia-common-430 libnvidia-common-440 libnvidia-decode-440
  libnvidia-encode-440 libnvidia-fbc1-440 libnvidia-gl-440 libnvidia-ifr1-440
  libpam-systemd libpolkit-agent-1-0 libpolkit-backend-1-0
  libpolkit-gobject-1-0 libxfont2 libxi-dev libxkbfile1 libxmu-dev
  libxmu-headers libxnvctrl0 nsig

[отсюда](https://lenta.ru/news/2008/11/13/medialogy/) список самых влиятельных СМИ:
* ["Газета.Ру"](https://www.gazeta.ru) `https://www.gazeta.ru`
* [РБК Daily.ru](https://www.rbc.ru) `https://www.rbc.ru`
* ["Лента.Ру"](https://lenta.ru) `https://lenta.ru`
* Quote.ru -- тот же РБК
* ["Взгляд"](https://vz.ru/) `https://vz.ru/`


In [0]:
# формирует урл поискового запроса в гугле
def format_google_url(search_string, dates_range_min, dates_range_max, site):
    """given params, returns google search url"""
    result =  f"https://www.google.com/search?q={search_string}" +\
              f"%20site:{site}&num=20&cad=h&tbs=cdr:1," +\
              f"cd_min:{dates_range_min},cd_max:{dates_range_max}"
    
    return result

# формирует урл поискового запроса в гугле
def google_search_references_links(gsearch_url):
    """given google search url, extracts search results links"""
    found_links = !(lynx --dump {gsearch_url}) \
                   | sed -n '/^References$/,/^   Hidden links:$/p' \
                   | grep "https://www.google.com/url?q=" \
                   | column -t | sed s"|^.*https://||" | sed s"|&.*$||"
    return found_links

In [0]:
persons = ["навальный", "путин"]
dates_range = {"date_min": "01.01.2018",
               "date_max": "01.04.2018"}
sites = ["gazeta.ru", "rbc.ru" , "lenta.ru", "vz.ru"]


from collections import defaultdict as dd
social_media_links = dd(lambda :list())
for site in sites:
    for person in persons:
        person_at_site_gsearch_url = format_google_url(person, 
                                                       dates_range["date_min"],
                                                       dates_range["date_max"], 
                                                       site)
        person_at_site_found_links = google_search_references_links(person_at_site_gsearch_url)
        social_media_links[site].extend(person_at_site_found_links)

for site, site_links in social_media_links.items():
    social_media_links[site] = sorted(site_links)


In [5]:
import pandas as pd

smi_urls_examples = pd.DataFrame([{"source":key, "url":value} for 
              key in social_media_links.keys()
              for value in social_media_links[key]])
smi_urls_examples.describe()

,source,url
count,84,84
unique,4,84
top,gazeta.ru,www.rbc.ru/rbcfreenews/5def85c09a7947399e0a1d82
freq,21,1


In [6]:
import IPython.display

IPython.display.HTML(smi_urls_examples[["url", "source"]]
                     [~ smi_urls_examples.url.str.contains("www.google.com")].to_html())

,url,source
0,www.gazeta.ru/lifestyle/style/2019/10/a_12774668.shtml,gazeta.ru
1,www.gazeta.ru/news/seealso/3335741.shtml,gazeta.ru
2,www.gazeta.ru/politics/2019/12/04_a_12847892.shtml,gazeta.ru
3,www.gazeta.ru/politics/2019/12/10_a_12856898.shtml,gazeta.ru
4,www.gazeta.ru/politics/elections2013/c5553693.shtml,gazeta.ru
5,www.gazeta.ru/politics/infographics/politicsbiografiya_alekseya_navalnogo.shtml,gazeta.ru
6,www.gazeta.ru/politics/news/2019/07/24/n_13256683.shtml,gazeta.ru
7,www.gazeta.ru/politics/news/2019/10/23/n_13612784.shtml,gazeta.ru
8,www.gazeta.ru/politics/news/2019/11/13/n_13693040.shtml,gazeta.ru
9,www.gazeta.ru/politics/news/2019/11/20/n_13719944.shtml,gazeta.ru


Все нашедшиеся ссылки сформированы по каким-то паттернам.

в lenta.ru это `lenta.ru/{news, articles}/{year}/{month}/{day}/..` .  
Методом тыка также установлено, что по адресу `lenta.ru/{news, articles}/{year}/{month}/{day}` можно найти все записи Ленты за день. 

в gazeta.ru это 
`gazeta.ru/{politics, social}/news/{year}/{month}/{day}/..` .  
Методом пристального взгляда также установлено, что на сайте газеты есть форма поиска новостей по запросу и датам, она генерирует поисковый урл такго формата: `gazeta.ru/search.shtml?p=search&page={page_num}&text={query}&from={day}.{month}.{year}&to={day}.{month}.{year}&sort_order=published_desc&input=utf8`

в rbc это
`rbc.ru/{politics, society}/{day}/{month}/{year}/..` 
и
`rbc.ru/rbcfreenews/..` .  
В рбк нашелся какой-то тег. Методом пристального взгляда также установлено, что это тег про Навального и на странице тега есть удобный материал поиска по датам. Взгляд на новость про Путина позволяет найти тег про него: `rbc.ru/tags/?tag=Владимир%20Путин`

в vz это 
`vz.ru/{politics, society, news}/{year}/{month}/{day}/..` .  
Среди найденных урлов есть какой-то тег (Навальный), но механизм тегов vz не выглядит прозрачным -- непонятно, например, как найти тег про Путина.

---

**Примерно везде ссылка на страничку с новостью сформулирована по одному и тому же правилу** `АдресИздания/Категория/Дата/..` . Различаются тонкости реализации.

Краулинг можно организовать либо используя поисковый инструмент сайта (РБК, Газета), либо хотя бы перебирая новости за интересные даты (Лента). Чуть сложнее искать по vz.ru.

Давайте покраулим и немножко поисследуем Газету: РБК и Лента работают вроде без нареканий, краулить их скучно, а вот поиск у Газеты, оказывается, не так уж и хорош, кстати. Надо будет с этим разобраться (это и VZ поможет скраулить)

#### Краулим газету через её внутренний поиск

In [0]:
import requests
from bs4 import BeautifulSoup

default_date_min = {"day": "01", "month": "01", "year": "2018"}
default_date_max = {"day": "01", "month": "04", "year": "2018"}

def gazeta_ru_search(query, date_min=default_date_min, date_max = default_date_max):
    page_num = 0

    interesting_links = []

    while True:
        res= requests.get(f"https://www.gazeta.ru/search.shtml?p=search"+
                        f"&page={page_num}&text={query}"+
                        f"&from={date_min['day']}.{date_min['month']}.{date_min['year']}"+
                            f"&to={date_max['day']}.{date_max['month']}.{date_max['year']}"+
                        "&sort_order=published_desc&input=utf8")

        if str(res.status_code)[0] != '2':
            break
        
        soup = BeautifulSoup(res.content)
        found_links = []

        for link in soup.findAll('a'):
            possible_link = link.get('href')
            if "/politics/news/2018" in possible_link \
            or "/social/news/2018" in possible_link:
                found_links.append("https://www.gazeta.ru" + possible_link)

        if found_links:
            interesting_links.extend(found_links)
        else:
            break
        
        page_num += 1
    
    return interesting_links

поищем на Газете всё про Путина за начало 2018 года: там были выборы. Запишем все найденные ссылки на новости в файлик `INTERESTING_LINKS_FILENAME`

In [0]:
INTERESTING_LINKS_FILENAME = "interesting_links.txt"
with open(INTERESTING_LINKS_FILENAME, 'w', encoding="utf-8") as f:
    for interesting_link in gazeta_ru_search(query = "владимир%20путин"):
        print(interesting_link, file=f)

Скачаем каждую записанную страничку, используя консольный веб-браузер `lynx`

In [0]:
# эта ячейка создаёт папку crawl_results
# в ней будем хранить скачанные странички
!mkdir crawl_results

In [10]:
#@title ######`в этой ячейке спрятался файл с краулером gazeta_search_crawler_worker.py. правый клик мышью -> Form его покажет`
# содержимое этой ячейки запишется в файл gazeta_search_crawler_worker.py
%%writefile gazeta_search_crawler_worker.py

from IPython import get_ipython
import sys

links_fn, worker_num, workers_qtty = sys.argv[1:]
worker_num = int(worker_num)
worker_num -= 1  # from humans' to pytonic numeration
workers_qtty = int(workers_qtty)

with open(links_fn, 'r', encoding="utf-8") as f:
    interesting_links = [line.strip() 
                         for line_idx, line in enumerate(f) 
                         if line_idx % workers_qtty == worker_num]

for idx, link in enumerate(interesting_links):
    
    # скачиваем текст странички в файл
    get_ipython().system('lynx -nolist --dump {link}' + \
                         ' > crawl_results/{idx*workers_qtty + worker_num}_res.txt')
    
    # просто логгирование
    get_ipython().system('echo $(date)' + \
                         ' worker {worker_num + 1} of {workers_qtty}' + \
                         ' line {idx} of {len(interesting_links)}')




Writing gazeta_search_crawler_worker.py


Запустим в фоновом режиме 5 краулеров, так побыстрее.

In [11]:
number_of_workers = 5
for worker_num in range(1, number_of_workers+1):
    # nohup -- способ запустить какую-то команду в фоновом режиме
    # >> crawler.log -- просьба дописывать весь вывод команды в файл crawler.log
    !nohup \
     ipython3 gazeta_search_crawler_worker.py \
     {INTERESTING_LINKS_FILENAME} {worker_num} {number_of_workers} \
     >> crawler.log &

nohup: redirecting stderr to stdout
nohup: redirecting stderr to stdout
nohup: redirecting stderr to stdout
nohup: redirecting stderr to stdout
nohup: redirecting stderr to stdout


In [12]:
import time

!ps ax | grep crawler

while True:
    # команда !ps ax | grep crawler 
    # ('!' -- синтаксис тетрадок, сама команда справа от '!')
    # это способ увидеть запущенные задачи, содержащие в себе слово crawler
    # первая колонка вывода -- число-идентификатор запущенной задачи
    # зная это число, задачу можно убить командой kill -2 это_число
    # или совсем убить командой kill -9 это_число
    processes_ran = !ps ax | grep crawler
    if len(processes_ran) < number_of_workers:
        break
    else:
        time.sleep(5)

    632 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/ipython3 gazeta_search_crawler_worker.py interesting_links.txt 1 5
    643 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/ipython3 gazeta_search_crawler_worker.py interesting_links.txt 2 5
    682 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/ipython3 gazeta_search_crawler_worker.py interesting_links.txt 3 5
    714 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/ipython3 gazeta_search_crawler_worker.py interesting_links.txt 4 5
    781 ?        Sl     0:01 /usr/bin/python3 /usr/local/bin/ipython3 gazeta_search_crawler_worker.py interesting_links.txt 5 5
    861 ?        S      0:00 /bin/bash -c ps ax | grep crawler
    863 ?        S      0:00 grep crawler


In [13]:
!head crawl_results/*

==> crawl_results/0_res.txt <==
   #alternate

   IFRAME: https://www.googletagmanager.com/ns.html?id=GTM-KJBSQR

   Top.Mail.Ru
   Top-100

Подпишитесь на «Газету.Ru»

   Самое актуальное и интересное на вашей электронной почте

==> crawl_results/1000_res.txt <==
   #alternate

   IFRAME: https://www.googletagmanager.com/ns.html?id=GTM-KJBSQR

   Top.Mail.Ru
   Top-100

Подпишитесь на «Газету.Ru»

   Самое актуальное и интересное на вашей электронной почте

==> crawl_results/1001_res.txt <==
   #alternate

   IFRAME: https://www.googletagmanager.com/ns.html?id=GTM-KJBSQR

   Top.Mail.Ru
   Top-100

Подпишитесь на «Газету.Ru»

   Самое актуальное и интересное на вашей электронной почте

==> crawl_results/1002_res.txt <==
   #alternate

   IFRAME: https://www.googletagmanager.com/ns.html?id=GTM-KJBSQR

   Top.Mail.Ru
   Top-100

Подпишитесь на «Газету.Ru»

   Самое актуальное и интересное на вашей электронной почте

==> crawl_results/1003_res.txt <==
   #alternate

   IFRAME: https://ww

Скрауленные странички, похоже, имеют общие детали интерфейса

In [14]:
!diff crawl_results/93_res.txt crawl_results/167_res.txt

33,40c33,42
<    Внешняя политика
<    В мире
<    Власть
<    Регионы
<    Партии
<    Исследование
<    Оппозиция
<    Скандал
---
>    Город
>    Религия
>    Происшествия
>    Власть и право
>    Криминал
>    Природа
>    Образование
>    Технологии
>    Здоровье
>    Погода
42c44
< Путин внес в Госдуму законопроект о контроле в противодействии коррупции
---
> У консульства России в Сиэтле почтили память погибших в Кемерово
44c46
<    01.04.2018 | 16:46
---
>    29.03.2018 | 06:48
47,52c49,54
<    Президент России Владимир Путин и председатель Следственного комитета
<    РФ Александр Бастрыкин, 28... Президент России Владимир Путин и
<    председатель Следственного комитета РФ Александр Бастрыкин, 28...
<    Президент России Владимир Путин и председатель Следственного комитета
<    РФ Александр Бастрыкин, 28...
<    Михаил Климентьев/ТАСС
---
>    Во время акции памяти жертв пожара в Кемерово на Пушкинской площади в
>    Москве, 27 марта 2018 года Во время акции памяти жертв пожар

In [15]:
!cat crawl_results/45_res.txt

   #alternate

   IFRAME: https://www.googletagmanager.com/ns.html?id=GTM-KJBSQR

   Top.Mail.Ru
   Top-100

Подпишитесь на «Газету.Ru»

   Самое актуальное и интересное на вашей электронной почте

   IFRAME: https://subscriptions.rambler.ru/subscribe_form/gazeta:header/

   Курсы валют $ 63.63 (+0.054) € 70.48 (+0.189)
   в Москве: [cloudy.svg] 3°C
   Газета.Ru

   Новости
   Политика
   Бизнес
   Общество
   Армия
   Мнения
   Культура
   Наука
   Технологии
   Авто
   Стиль
   Спорт
   Поисковый запрос ____________________ Найти → search______________
   pt__________________

   Город
   Религия
   Происшествия
   Власть и право
   Криминал
   Природа
   Образование
   Технологии
   Здоровье
   Погода

Бывшая глава госстройнадзора Кузбасса арестована после пожара в Кемерово

   31.03.2018 | 12:48

   close
   Пожар в ТЦ «Зимняя вишня» в Кемерово, 25 марта 2018 года Пожар в ТЦ
   «Зимняя вишня» в Кемерово, 25 марта 2018 года Пожар в ТЦ «Зимняя вишня»
   в Кемерово, 25 марта 2018 года

Интересный нам текст находится между последовательностями звёздочек.

Вообще, конечно, это всё меняется от издания к изданию, но в каждом издании все новости находятся на похоже оформленных страничках, различающихся по сути только содержанием, собственно, новости. 

Ленивый NLP-аналитик обойдётся $\mathit{tf-idf}$ весами, посчитанными на сыром файле странички, и не особенно потеряет на этой лени.

Но если мы краулим какую-то газету (или десяти газет), не жалко для каждой потратить 5 минут на поиск общего паттерна и ещё чуть -- на программирование его аккуратной очистки.

In [0]:
import pandas as pd
import re
def parse_gazeta_crawl_results(crawl_results_filenames):
    gazeta_search_articles_crawled = []

    date_regex = r"\d{2}.\d{2}.\d{4}"
    time_regex = r"\d{2}\:\d{2}"
    datetime_regex = date_regex + r"\s\|\s" + time_regex

    for crawl_result_idx, crawl_result_fn in enumerate(crawl_results_filenames):

        crawl_result_fpath = "crawl_results/" + crawl_result_fn
        with open(crawl_result_fpath, 'r', encoding="utf-8") as crawl_result_f:
            crawl_result_text = [line.strip()
                                for line in crawl_result_f.readlines() 
                                if line.strip() 
                                and not line.strip().startswith("IFRAME")]
        
        #region article date extraction
        article_datetime = None
        for line in crawl_result_text:
            if re.match(datetime_regex, line):
                article_datetime = line
                break
        assert article_datetime is not None
        article_date, article_time = article_datetime.split('|')
        #endregion

        #region article actual text extraction
        #the text is between stars
        article_text_tmp = ' '.join(crawl_result_text)
        article_text = article_text_tmp[article_text_tmp.index("* "*5):][10:]
        article_text = article_text[:article_text.index("* "*5)]
        article_text = article_text.strip()
        article_text = article_text.strip("Комментировать.")
        article_text = article_text.strip("Комментировать")
        #endregion

        gazeta_search_articles_crawled.append({"date": article_date.strip(), 
                                            "time": article_time.strip(), 
                                            "text": article_text.strip()})
        
    return gazeta_search_articles_crawled

In [17]:
crawl_results_filenames = !ls -1 crawl_results/

gazeta_search_articles_df = pd.DataFrame(parse_gazeta_crawl_results(crawl_results_filenames))
gazeta_search_articles_df.head()

,date,time,text
0,01.04.2018,20:56,Сенатор Владимир Джабаров оценил намерения пос...
1,11.03.2018,18:57,"Президент России Владимир Путин заявил, что не..."
2,11.03.2018,18:33,"Премьер-министр Дмитрий Медведев рассказал, чт..."
3,11.03.2018,18:33,"Премьер-министр Дмитрий Медведев рассказал, чт..."
4,11.03.2018,18:33,"Премьер-министр Дмитрий Медведев рассказал, чт..."


In [18]:
print(gazeta_search_articles_df.text.str.wrap(60)[30])

Президент России Владимир Путин рассказал, как за час до
открытия Олимпиады в 2014 году ему позвонили и сообщили о
захвате турецкого самолета, который требовали направить в
Сочи. Об этом он сказал в фильме Андрея Кондрашова,
выложенном в «Одноклассниках». По его словам, звонок
поступил, когда глава государства и представители МОК ехали
в автобусе на стадион «Фишт». «Где-то на середине пути
адъютант мне передал телефон, позвонил один из руководителей
оперативного штаба по обеспечению безопасности Олимпийских
игр. И докладывает: захвачен самолет, который должен
следовать из Украины в Стамбул, захватчики самолета требуют
посадки в Сочи», — рассказал Путин. Глава государства
отметил, что было принято решение действовать согласно
плану, предусмотренному для такого развития событий, и сбить
самолет. «Раздался второй звонок, и тот же самый
руководитель штаба по линии безопасности сказал, что это
была пьяная выходка, самолет направляется в Турцию и скоро
там садится», — добавил Путин. На вопро

In [19]:
!pip -qq install natasha
!pip -qq install yargy
!pip -qq install ipymarkup



     |████████████████████████████████| 778kB 8.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 7.1MB 47.7MB/s 


In [0]:
from natasha import NamesExtractor

extractor = NamesExtractor() 

def extract_names(text):
    matches = extractor(text)
    return [(_.span, _.fact) for _ in matches]



In [0]:
gazeta_search_articles_df["names"] = gazeta_search_articles_df.text.apply(extract_names)

In [22]:
from collections import Counter

Counter([elem[1].last 
         for row in gazeta_search_articles_df.names 
         for elem in row]).most_common()[:20]

[('путин', 2983),
 (None, 1185),
 ('тулеев', 256),
 ('песок', 208),
 ('трамп', 179),
 ('грудинин', 123),
 ('жириновский', 120),
 ('собчак', 99),
 ('кондрашов', 84),
 ('явлинский', 78),
 ('бабурин', 75),
 ('сурайкин', 75),
 ('титов', 69),
 ('цивилев', 67),
 ('бастрыкин', 57),
 ('меркель', 53),
 ('маккейн', 32),
 ('володин', 30),
 ('медведев', 29),
 ('мегин', 27)]

In [0]:
gazeta_search_articles_df["last_names"] = gazeta_search_articles_df.names.apply(
    lambda x: list({elem[1].last for elem in x if elem[1].last is not None}))



In [0]:
persons = {elem for row in gazeta_search_articles_df.last_names for elem in row}
cooccurrence_dict = {}
for person1 in persons:
    cooccurrence_dict[person1] = {}
    for person2 in persons:
        cooccurrence_dict[person1][person2] = {'weight': 0}

for row in gazeta_search_articles_df.last_names:
    for person1_idx, person1 in enumerate(row):
        for person2 in row[person1_idx+1:]:
            cooccurrence_dict[person1][person2]['weight'] += 1

In [0]:
import networkx as nx
nx_graph = nx.from_dict_of_dicts(cooccurrence_dict)


In [26]:
sorted([elem for elem in nx_graph.edges(data=True) if elem[2]['weight'] > 0],
       key= lambda x: x[2]['weight'],reverse=True)[:20]


[('путин', 'песок', {'weight': 184}),
 ('грудинин', 'путин', {'weight': 84}),
 ('сурайкин', 'путин', {'weight': 66}),
 ('сурайкин', 'жириновский', {'weight': 66}),
 ('явлинский', 'путин', {'weight': 66}),
 ('явлинский', 'жириновский', {'weight': 66}),
 ('бабурин', 'жириновский', {'weight': 66}),
 ('грудинин', 'титов', {'weight': 63}),
 ('сурайкин', 'собчак', {'weight': 63}),
 ('сурайкин', 'титов', {'weight': 63}),
 ('явлинский', 'собчак', {'weight': 63}),
 ('явлинский', 'титов', {'weight': 63}),
 ('бабурин', 'собчак', {'weight': 63}),
 ('бабурин', 'титов', {'weight': 63}),
 ('собчак', 'жириновский', {'weight': 63}),
 ('собчак', 'титов', {'weight': 63}),
 ('жириновский', 'титов', {'weight': 63}),
 ('сурайкин', 'явлинский', {'weight': 45}),
 ('бастрыкин', 'путин', {'weight': 27}),
 ('путин', 'пескова', {'weight': 27})]

**TODO: надо будет на паре по всяким визуализациям порисовать этот ^ граф.**

In [27]:
from ufal.udpipe import Model, Pipeline
UDPIPE_MODEL_FN = "model_ru.udpipe"
!wget -O {UDPIPE_MODEL_FN} https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true

--2019-12-10 13:46:24--  https://github.com/jwijffels/udpipe.models.ud.2.0/blob/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe?raw=true
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2019-12-10 13:46:24--  https://github.com/jwijffels/udpipe.models.ud.2.0/raw/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe [following]
--2019-12-10 13:46:25--  https://raw.githubusercontent.com/jwijffels/udpipe.models.ud.2.0/master/inst/udpipe-ud-2.0-170801/russian-ud-2.0-170801.udpipe
Re

In [0]:
from nltk import DependencyGraph

model = Model.load(UDPIPE_MODEL_FN)
pipeline = Pipeline(model, 'generic_tokenizer', '', '', '')

def get_actions(text):
    parsed = pipeline.process(text)


    actions = []
    for sent in parsed.split('\n\n'):
        # убираем коменты
        sent = '\n'.join([line for line in sent.split('\n') if not line.startswith('#')])
        # заменяем регистр для root
        sent = sent.replace('\troot\t', '\tROOT\t')
        graph=DependencyGraph(tree_str=sent)
        try:
            actions = [elem for elem in graph.triples() if elem[1] == 'nsubj']
            pretty_actions = []
            for action in actions:
                if action[2][1] in ['NOUN', 'PROPN'] and action[0][1] == 'VERB':
                    action_verb = action[0][0]
                    actor = action[2][0]
                    pretty_actions.append((actor, action_verb))
                else:
                    action_verb = None
                    actor = None
                    pretty_actions.append(action)
                    
        except:
            pass
    
    return pretty_actions


In [0]:
gazeta_search_articles_df["actions"] = gazeta_search_articles_df.text.apply(get_actions)

In [34]:
Counter([elem for row in gazeta_search_articles_df["actions"] for elem in row if len(elem) == 2]).most_common()[:20]


[(('Путин', 'рассказал'), 48),
 (('Путин', 'заявил'), 20),
 (('Газета', 'ведет'), 18),
 (('Владимир', 'заявил'), 12),
 (('Путин', 'признался'), 12),
 (('Президент', 'отметил'), 10),
 (('изменения', 'произойдут'), 9),
 (('президент', 'рассказал'), 8),
 (('Цивилев', 'отметил'), 8),
 (('десятки', 'погибли'), 8),
 (('дети', 'погибли'), 8),
 (('Владимир', 'рассказал'), 6),
 (('заместитель', 'назначен'), 6),
 (('Тулеев', 'подал'), 6),
 (('Владимир', 'назвал'), 6),
 (('президент', 'прочитал'), 6),
 (('Тул', 'прокомментировал'), 6),
 (('новости', 'упал'), 6),
 (('Путин', 'назначил'), 6),
 (('Путин', 'подписал'), 6)]